# PROVIDE SAMPLE SIZE AND PICKLE FILENAMES

In [ ]:
# number of MIDI files used from Maestro dataset
SAMPLE_SIZE = 100

SAMPLE_FILE = 'pickles/sampled_100_midi.pkl'
NOTE_TOKENIZER_FILE = 'pickles/note_tokenizer_100_samples.pkl'
UNIQUE_NOTES_FILE = 'pickles/unique_notes_100_samples.pkl'

# Environment Setup

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/Shareddrives/295/Code/RNN_Transformer')

In [ ]:
!pip install -r requirements.txt

     |████████████████████████████████| 5.6 MB 10.8 MB/s 
     |████████████████████████████████| 516.2 MB 13 kB/s 
     |████████████████████████████████| 69 kB 7.4 MB/s 
     |████████████████████████████████| 562 kB 50.0 MB/s 
     |████████████████████████████████| 14.5 MB 43.4 MB/s 
     |████████████████████████████████| 454 kB 65.1 MB/s 
     |████████████████████████████████| 2.9 MB 37.5 MB/s 
     |████████████████████████████████| 3.0 MB 38.1 MB/s 
     |████████████████████████████████| 51 kB 6.8 MB/s 
  Created wheel for folium: filename=folium-0.2.1-py3-none-any.whl size=79808 sha256=9bb2922afe3e5c0804b66edb177f06a3a5e75a160f11bc4e0afd3c74d3fc578d
  Stored in directory: /root/.cache/pip/wheels/9a/f0/3a/3f79a6914ff5affaf50cabad60c9f4d565283283c97f0bdccf
  Created wheel for imgaug: filename=imgaug-0.2.5-py3-none-any.whl size=561438 sha256=8209414897798c9187bd2f98add573d42fb5661480bac053e45c785a43a9b065
  Stored in directory: /root/.cache/pip/wheels/60/dd/38/d1dc2cad2b6a66dc0

In [ ]:
# !sudo apt-get install python3.8
# !pip install -q library
# !pip install pretty_midi
# !pip install --upgrade pip
# !pip install tensorflow==2.2.0rc4
# !pip install folium==0.2.1
# !pip install imgaug==0.2.5
# !pip install -U numpy==1.19.2

In [ ]:
import tensorflow as tf
from tensorflow.keras import backend as K
import glob
import random
import pretty_midi
import IPython
import numpy as np
from tqdm.notebook import tnrange, tqdm_notebook, tqdm
from random import shuffle, seed
import numpy as np
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Nadam
import numpy as np
from numpy.random import choice
import pickle
import matplotlib.pyplot as plt

import unicodedata
import re
import numpy as np
import os
import io
import time
import pickle

In [ ]:
!wAget https://storage.googleapis.com/magentadata/datasets/maestro/v1.0.0/maestro-v1.0.0-midi.zip

In [ ]:
!unzip maestro-v1.0.0-midi.zip

# Get All Midi Files

In [ ]:
!pwd

/content/drive/Shareddrives/295/Code/RNN_Transformer


In [ ]:
def get_list_midi(folder = 'maestro-v1.0.0/**/*.midi', seed_int = 666):
  list_all_midi = glob.glob(folder)
  seed(seed_int)
  shuffle(list_all_midi)
  return list_all_midi

list_all_midi = get_list_midi()

In [ ]:
list_all_midi[:10]

['maestro-v1.0.0/2006/MIDI-Unprocessed_10_R1_2006_01-04_ORIG_MID--AUDIO_10_R1_2006_03_Track03_wav.midi',
 'maestro-v1.0.0/2008/MIDI-Unprocessed_07_R1_2008_01-04_ORIG_MID--AUDIO_07_R1_2008_wav--3.midi',
 'maestro-v1.0.0/2017/MIDI-Unprocessed_047_PIANO047_MID--AUDIO-split_07-06-17_Piano-e_2-04_wav--4.midi',
 'maestro-v1.0.0/2011/MIDI-Unprocessed_08_R1_2011_MID--AUDIO_R1-D3_08_Track08_wav.midi',
 'maestro-v1.0.0/2008/MIDI-Unprocessed_09_R1_2008_01-05_ORIG_MID--AUDIO_09_R1_2008_wav--3.midi',
 'maestro-v1.0.0/2015/MIDI-Unprocessed_R2_D2-12-13-15_mid--AUDIO-from_mp3_12_R2_2015_wav--2.midi',
 'maestro-v1.0.0/2015/MIDI-Unprocessed_R1_D2-13-20_mid--AUDIO-from_mp3_18_R1_2015_wav--1.midi',
 'maestro-v1.0.0/2011/MIDI-Unprocessed_06_R1_2011_MID--AUDIO_R1-D2_17_Track17_wav.midi',
 'maestro-v1.0.0/2008/MIDI-Unprocessed_07_R1_2008_01-04_ORIG_MID--AUDIO_07_R1_2008_wav--2.midi',
 'maestro-v1.0.0/2017/MIDI-Unprocessed_075_PIANO075_MID--AUDIO-split_07-08-17_Piano-e_2-06_wav--3.midi']

# Prepare some functions
See its documentation to see the function

In [ ]:
class NoteTokenizer:
    
    def __init__(self):
      self.notes_to_index = {}
      self.index_to_notes = {}
      self.num_of_word = 0
      self.unique_word = 0
      self.notes_freq = {}
        
    def transform(self,list_array):
      transformed_list = []
      for instance in list_array:
          transformed_list.append([self.notes_to_index[note] for note in instance])
      return np.array(transformed_list, dtype=np.int32)
 
    def partial_fit(self, notes):
        for note in notes:
            note_str = ','.join(str(a) for a in note)
            if note_str in self.notes_freq:
                self.notes_freq[note_str] += 1
                self.num_of_word += 1
            else:
                self.notes_freq[note_str] = 1
                self.unique_word += 1
                self.num_of_word += 1
                self.notes_to_index[note_str], self.index_to_notes[self.unique_word] = self.unique_word, note_str
            
    def add_new_note(self, note):
        assert note not in self.notes_to_index
        self.unique_word += 1
        self.notes_to_index[note], self.index_to_notes[self.unique_word] = self.unique_word, note
        
def generate_batch_song(list_all_midi, batch_music=16, start_index=0, fs=30, seq_len=50, use_tqdm=False):
    assert len(list_all_midi) >= batch_music
    dict_time_notes = generate_dict_time_notes(list_all_midi, batch_music, start_index, fs, use_tqdm=use_tqdm)
    
    list_musics = process_notes_in_song(dict_time_notes, seq_len)
    collected_list_input, collected_list_target = [], []
     
    for music in list_musics:
        list_training, list_target = generate_input_and_target(music, seq_len)
        collected_list_input += list_training
        collected_list_target += list_target
    return collected_list_input, collected_list_target

def generate_dict_time_notes(list_all_midi, batch_song = 16, start_index=0, fs=30, use_tqdm=True):
    assert len(list_all_midi) >= batch_song
    
    dict_time_notes = {}
    process_tqdm_midi = tqdm_notebook(range(start_index, min(start_index + batch_song, len(list_all_midi)))) if use_tqdm else range(start_index,  min(start_index + batch_song, len(list_all_midi)))
    for i in process_tqdm_midi:
        midi_file_name = list_all_midi[i]
        if use_tqdm:
            process_tqdm_midi.set_description("Processing {}".format(midi_file_name))
        try: # Handle exception on malformat MIDI files
            midi_pretty_format = pretty_midi.PrettyMIDI(midi_file_name)
            print(midi_pretty_format)
            piano_midi = midi_pretty_format.instruments[0] # Get the piano channels
            print(piano_midi)
            piano_roll = piano_midi.get_piano_roll(fs=fs)
            print(piano_roll, piano_roll.shape)
            dict_time_notes[i] = piano_roll
        except Exception as e:
            print(e)
            print("broken file : {}".format(midi_file_name))
            pass
    return dict_time_notes

def generate_input_and_target(dict_keys_time, seq_len=50):
    # Get the start time and end time
    start_time, end_time = list(dict_keys_time.keys())[0], list(dict_keys_time.keys())[-1]
    list_training, list_target = [], []
    for index_enum, time in enumerate(range(start_time, end_time)):
        list_append_training, list_append_target = [], []
        start_iterate = 0
        flag_target_append = False # flag to append the test list
        if index_enum < seq_len:
            start_iterate = seq_len - index_enum - 1
            for i in range(start_iterate): # add 'e' to the seq list. 
                list_append_training.append('e')
                flag_target_append = True

        for i in range(start_iterate,seq_len):
            index_enum = time - (seq_len - i - 1)
            if index_enum in dict_keys_time:
                list_append_training.append(','.join(str(x) for x in dict_keys_time[index_enum]))      
            else:
                list_append_training.append('e')

        # add time + 1 to the list_append_target
        if time+1 in dict_keys_time:
            list_append_target.append(','.join(str(x) for x in dict_keys_time[time+1]))
        else:
            list_append_target.append('e')
        list_training.append(list_append_training)
        list_target.append(list_append_target)
    return list_training, list_target

def process_notes_in_song(dict_time_notes, seq_len = 50):
    list_of_dict_keys_time = []
    
    for key in dict_time_notes:
        sample = dict_time_notes[key]
        times = np.unique(np.where(sample > 0)[1])
        index = np.where(sample > 0)
        dict_keys_time = {}

        for time in times:
            index_where = np.where(index[1] == time)
            notes = index[0][index_where]
            dict_keys_time[time] = notes
        list_of_dict_keys_time.append(dict_keys_time)
    return list_of_dict_keys_time



# Sample 100 midi files from the datasets
These midi files will be used to train the neural network

In [ ]:
# Get 200 midis file from the datasets.. 

sampled_200_midi = list_all_midi[0:SAMPLE_SIZE] 

In [ ]:
file = open(SAMPLE_FILE, 'wb')
pickle.dump(sampled_200_midi, file)

We create a map of note -> index here using NoteTokenizer that we've defined before.  
This object will be used to transform the list of notes to be ready for the input of Neural Network

In [ ]:
batch = 1
start_index = 0
note_tokenizer = NoteTokenizer()

for i in tqdm_notebook(range(len(sampled_200_midi))):
    print(sampled_200_midi[i])
    dict_time_notes = generate_dict_time_notes(sampled_200_midi, batch_song=1, start_index=i, use_tqdm=False, fs=5)
    full_notes = process_notes_in_song(dict_time_notes)
    for note in full_notes:
        note_tokenizer.partial_fit(list(note.values()))
   

  0%|          | 0/100 [00:00<?, ?it/s]

maestro-v1.0.0/2006/MIDI-Unprocessed_10_R1_2006_01-04_ORIG_MID--AUDIO_10_R1_2006_03_Track03_wav.midi
Instrument(program=0, is_drum=False, name="")
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] (128, 6315)
maestro-v1.0.0/2008/MIDI-Unprocessed_07_R1_2008_01-04_ORIG_MID--AUDIO_07_R1_2008_wav--3.midi
Instrument(program=0, is_drum=False, name="")
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] (128, 1285)
maestro-v1.0.0/2017/MIDI-Unprocessed_047_PIANO047_MID--AUDIO-split_07-06-17_Piano-e_2-04_wav--4.midi
Instrument(program=0, is_drum=False, name="")
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] (128, 2520)
maestro-v1.0.0/2011/MIDI-Unprocessed_08_R1_2011_MID--AUDIO_R1-D

In [ ]:
note_tokenizer.add_new_note('e') # Add empty notes

In [ ]:
unique_notes = note_tokenizer.unique_word
print(unique_notes)

60792


In [ ]:
file = open(UNIQUE_NOTES_FILE, 'wb')
pickle.dump(unique_notes, file)

In [ ]:
note_tokenizer

In [ ]:
file = open(NOTE_TOKENIZER_FILE, 'wb')
pickle.dump(note_tokenizer, file)

In [ ]:
dict_time_notes[99]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
full_notes

[{5: array([42, 49, 70]),
  6: array([42, 49, 70]),
  7: array([42, 49, 70]),
  8: array([42, 49, 58, 70]),
  9: array([42, 49, 58, 70]),
  10: array([42, 49, 58, 66, 70]),
  11: array([42, 49, 58, 66, 70]),
  12: array([42, 49, 58, 61, 66, 70]),
  13: array([42, 49, 58, 61, 66, 70]),
  14: array([42, 49, 58, 61, 66, 70]),
  15: array([42, 49, 58, 61, 66, 70]),
  16: array([42, 49, 58, 61, 66, 70]),
  17: array([42, 49, 58, 61, 66, 70]),
  18: array([42, 49, 58, 61, 66, 70]),
  19: array([42, 49, 58, 61, 66, 70]),
  20: array([42, 49, 58, 61, 66, 70]),
  21: array([42, 49, 58, 61, 66, 70]),
  22: array([42, 49, 58, 61, 66, 70]),
  23: array([42, 49, 58, 61, 66, 70]),
  24: array([42, 49, 58, 61, 66, 70]),
  25: array([42, 49, 58, 61, 66, 70]),
  26: array([39, 42, 49, 58, 61, 66, 70]),
  27: array([39, 42, 49, 51, 58, 61, 66, 70]),
  28: array([39, 51, 58, 70]),
  29: array([39, 51, 58, 70]),
  30: array([39, 51, 58, 63, 70]),
  31: array([39, 51, 58, 63, 70]),
  32: array([39, 51, 58,